In [49]:
import json
import pandas as pd
import gurobipy as gp
from datetime import datetime, timedelta

In [50]:
OBP_AVG = 0.327
SLG_AVG = 0.365

FPCT_AVG = {
    '2': 0.992,
    '3': 0.993,
    '4': 0.977, 
    '5': 0.929,
    '6': 0.964,
    '7': 0.977,
    '8': 0.988,
    '9': 0.976
}

POSITION_WEIGHT = {
    '2': {"batting": 0.95, "fielding": 1.05},
    '3': {"batting": 1.21, "fielding": 0.83},
    '4': {"batting": 0.97, "fielding": 1.03},
    '5': {"batting": 0.96, "fielding": 1.04},
    '6': {"batting": 0.94, "fielding": 1.06},
    '7': {"batting": 1.02, "fielding": 0.98},
    '8': {"batting": 1.04, "fielding": 0.96},
    '9': {"batting": 1.17, "fielding": 0.85},
}

OPPONENT_WEIGHT = {
    "中信兄弟": 1.15,
    "統一7-ELEVEn獅": 1.11,
    "樂天桃猿": 0.96, 
    "富邦悍將": 0.93,
    "味全龍": 0.85
}

dataRoot = "../clean/clean_all_"
playerRoot = "../球員異動/"

teamNameMap = {
    "中信兄弟": "brothers", 
    "味全龍": "dragons",
    "富邦悍將": "guardians", 
    "統一7-ELEVEn獅": "lions",
    "樂天桃猿": "monkeys"
}

### Import Data

In [51]:
class Player:
    def __init__(self, name, data):
        self.name = name
        self.data = data

        # A_{ij}
        canPlay = []
        for i in range(9):
            if (i + 1) in data["fielding"]["pos"]:
                canPlay.append(1)
            else:
                canPlay.append(0)
        self.canPlay = canPlay

        # F_{ij}
        F = {}
        for i in data["fielding"]["pos"]:
            F[str(i)] = float(data["fielding"]["FPCT"][str(i)]) / FPCT_AVG[str(i)]
            if F[str(i)] == 1:
                F[str(i)] = 0.98
            if F[str(i)] < 0.8:
                F[str(i)] = 0.8
        self.F = F

In [52]:
allData = {}
for team in teamNameMap.keys():
    with open(dataRoot + team + ".json", encoding="utf-8") as f:
        temp = []
        for line in f:
            player = json.loads(line)
            temp.append(Player(player["name"], player["data"]))
        allData[teamNameMap[team]] = temp

brothers = allData["brothers"]
dragons = allData["dragons"]
guardians = allData["guardians"]
lions = allData["lions"]
monkeys = allData["monkeys"]

In [53]:
playerList = {}

for team in teamNameMap.keys():
    tempDict = pd.read_csv(playerRoot + team + ".csv", header=0, index_col=0).to_dict()
    tempDict = {
        datetime.strptime(key, "%Y-%m-%d").date(): list(filter(lambda d: pd.isna(d) == False, list(value.values())))
        for key, value in tempDict.items()
    }
    playerList[teamNameMap[team]] = tempDict

In [54]:
def createDate(mm, dd):
    return datetime(2022, mm, dd, 0, 0, 0).date()

### Self-defined Parameters

In [55]:
aTeam = "中信兄弟"
aMonth = "May"
aGame = [
    {
        "date": createDate(5, 3),
        "field": "新莊", 
        "oppo": "富邦悍將", 
        "pitcher": "陳鴻文"
    }, 
    {
        "date":createDate(5, 5), 
        "field": "澄清湖", 
        "oppo": "味全龍", 
        "pitcher": "王維中"
    }, 
    {
        "date": createDate(5, 6), 
        "field": "洲際", 
        "oppo": "統一7-ELEVEn獅", 
        "pitcher": "江辰晏"
    }, 
    {
        "date": createDate(5, 7), 
        "field": "洲際", 
        "oppo": "統一7-ELEVEn獅", 
        "pitcher": "李其峰"
    }, 
    {
        "date": createDate(5, 8), 
        "field": "洲際", 
        "oppo": "統一7-ELEVEn獅", 
        "pitcher": "邱浩鈞"
    }
]
aStar = ["許基宏", "江坤宇"]
aPotential = ["岳東華"]
aInjured = ["周思齊"]

### Optimization

In [60]:
# 比賽當天 27 人名單
def findPlayerList(team, date):
    minDelta = timedelta(days=100)
    playerDay = date
    for key in playerList[teamNameMap[aTeam]].keys():
        if date > key:
            break
        if key - date < minDelta:
            minDelta = key - date
            playerDay = key
    return playerList[teamNameMap[team]][playerDay]
        

In [67]:
# B_i
def calcOPS(OBP, SLG):
    return OBP / OBP_AVG + SLG / SLG_AVG - 1

def calcB(game, player):
    if game["date"].month == 4:
        aMonth = "Apr"
    if game["date"].month == 5:
        aMonth = "May"

    OPSseason = float(player.data["batting"]["season"]["OPS+"])
    if OPSseason > 160:
        OPSseason = 100
    if OPSseason < 0:
        OPSseason = 0
    OPSmonth = calcOPS(
        float(player.data["batting"]["month"][aMonth]["OBP"]), 
        float(player.data["batting"]["month"][aMonth]["TB"]) / float(player.data["batting"]["month"][aMonth]["AB"])
    )
    OPSfield = calcOPS(
        float(player.data["batting"]["field"][game["field"]]["OBP"]), 
        float(player.data["batting"]["field"][game["field"]]["TB"]) / float(player.data["batting"]["field"][game["field"]]["AB"])
    )
    OBPseason = float(player.data["batting"]["season"]["OBP"])
    if OBPseason == 0:
        OBPseason = 1
    OBPp = float(player.data["batting"]["vsP"]["data"][game["oppo"]][game["pitcher"]]) if player.data["batting"]["vsP"]["data"][game["oppo"]][game["pitcher"]] != '0.000' else OBPseason

    B = (OPSseason + OPSmonth + OPSfield + OBPp * 100 / OBPseason) / 400
    return B

def calcV(game, player, pos):
    B = calcB(game, player)
    return (POSITION_WEIGHT[str(i)]["batting"] * B + POSITION_WEIGHT[str(pos)]["fielding"] * player.F[str(pos)]) * player.canPlay

In [77]:
model = gp.Model("model")

x = []
for g in range(len(aGame)):
    todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
    singleGame = []
    for i in range(len(todayPlayerList)):
        singlePlayer = []
        for j in range(9):
            singlePlayer.append(model.addVar(lb=0, vtype="B", name="x" + str(g) + "-" + todayPlayerList[i] + "-" + str(j)))
        singleGame.append(singlePlayer)
    x.append(singleGame)

obj = model.addVar(lb = 0, vtype = "C", name = "w")

model.setObjective(obj, gp.GRB.MAXIMIZE)

sumAll = 0
for g in range(len(aGame)):
    todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
    for i in range(len(todayPlayerList)):
        todayPlayerObject = allData[teamNameMap[aTeam]]
        # print("today", todayPlayerList[i])
        # for obje in todayPlayerObject:
        #     print(obje.name)
        thisPlayer = list(filter(lambda d: d.name == todayPlayerList[i], todayPlayerObject))[0]
        for j in range(9):
            sumAll += OPPONENT_WEIGHT[aGame[g]["oppo"]] * calcV(aGame[g], thisPlayer, j) * x[g][i][j]

model.addConstr(obj == sumAll)

model.optimize()

IndexError: list index out of range